# Representación semántica y búsqueda
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

Cuando la información esta poco específicada, las palabras adecuadas podrían ser dificiles de tener o limitar. En estos casos, la representaciones semánticas que permiten buscar _lo que se desea_ por medio de conceptos nos acerca más a la posibilidad de obtener información útil.

Como anteriormente se presento, una de las representaciones semánticas más aceptadas son aquellas basadas en word embeddings. Recordando, estas son representaciones vectoriales de cada palabra, donde la semántica se asocia con la estructura en un espacio métrico, i.e., cercano en la métrica significa similar semánticamente.

# Representación semántica de documentos basada en _word embeddings_
Un documento puede verse como una bolsa de palabras, como en el modelado tradicional, pero en lugar de usar los términos como símbolos se pueden usar los vectores semánticos. El como usar esta nube de puntos multidimensional para obtener una representación computacionalmente manejable y a la vez eficaz, es un tema que ha llevado al desarrollo de modelos cada vez más complejos, como pueden ser los grandes modelos de lenguaje. 

La manera forma más directa de definir una representación semántica de documentos es el uso de centroides, esto es, un vector de la misma dimensión que sumariza a un conjunto de vectores usando su media geométrica. De manera más precisa, sea $E$ una matrix $m \times n$ de embeddings del vocabulario, i.e., considerando un vocabulario de tamaño $n$ y dimensión $m$.
Sea $D_{m,\ell}$ la submatriz de $E$ que contiene los $\ell$ vectores del documento $\textsf{doc}$. El prototipo $\vec{d}$ de $D$ esta definido de la siguiente forma:

\begin{equation}
\vec{d} = \frac{1}{\ell} \sum_{i=1}^\ell D_i
\end{equation}


Donde $D_i$ es el i-ésimo vector columna de $D$. Dado que se usa el coseno como similitud, también es factible el cálculo como la suma vectorial normalizada, esto es,

$$ \vec{d} = \frac{\sum_{i=1}^\ell D_i}{\lVert \sum_{i=1}^\ell D_i \rVert}$$

También es posible añadir información local basada en la representación de bolsa de plabras. Por ejemplo, realizar una suma pesada usando el peso de las palabras mediante la frecuencia normalizada de término (TF) o la probabilidad de término $pt$, ver unidad 3 para más información.

$$\vec{d} = \sum_{i=1}^{\ell} \textsf{TF}(t_i, \textsf{doc}) \cdot D_i$$

Donde $t_i$ es el $i$-ésimo término de $\textsf{doc}$. Note que el orden no se captura y solo es necesario para la notación.

## Usando la nube de puntos

La representación de nube de puntos puede usarse directamente sin realizar prototipos, sin embargo, es posible que dada su complejidad, dimensión intrinseca, tenga poca utilidad en grandes volumenes de información. Por ejemplo, es posible utilizar la distancia de [Hausdorff](https://en.wikipedia.org/wiki/Hausdorff_distance); que entre dos nubes de puntos (documentos para este caso) esta definida como sigue:

$$ H_d(U, V) = \max \left\{ \max_{u \in U} \min_{x \in V} d(u, x), \max_{v \in V} \min_{x \in U} d(v, x) \right\} $$

Donde $\textsf{nn}$ encuentra el vecino cercano del primer argumento en el segundo. Podemos ver la intuición de esta distancia, analizando las partes de la expresión. 
La diferencia entre ambos conjuntos se representa con el máximo de las distancias mínimas. Note que esta parte es idéntica a resolver una búsqueda de vecinos cercanos. La parte más externa de la expresión se repite para los dos conjuntos para preservar la simétria de la función.

De la misma forma, es posible obtener variaciones de interés como considerar todas las distancias cercanas en lugar de solo las máximas, lo cual puede reducir formas caprichosas de _outlayers_.

$$ H^+_d(U, V) = \max \left\{ \sum_{u \in U} \min_{x \in V} d(u, x), \sum_{v \in V} \min_{x \in U} d(v, x) \right\}$$

La información local de una representación de bolsa de palabras también puede ser aprovechada, por ejemplo, añadiendo información de $\textsf{TF}$

$$ H^\textsf{TF}_d(U, V) = \max \left\{ \sum_{u \in U} {\textsf{TF}(u, U) \min_{x \in V} d(u, x)}, \sum_{v \in V} {\textsf{TF}(v, V) \min_{x \in U} d(v, x)} \right\} $$

# Notas adicionales
La versión de $H_d$ que usa la probabilidad de término $\textsf{pt}$ es similar a la aproximación 2 listada en [@KSKW2015]. Los prototipos son similares a la aproximación 1 del mismo artículo. En el artículo se presenta la función de distancia _word mover's distance_ (WMD) que es una adecuación de la Earths mover's distance (EMD) [PW2009]. EMD es la solución óptima a un problema de transportación, la WMD es la adaptación teniendo en cuenta embeddings de palabras.

# Ejemplo


In [1]:

using Pkg
Pkg.activate(".")
!isfile("Manifest.toml") && Pkg.add([
    PackageSpec(name="SimilaritySearch", version="0.9"),
    PackageSpec(name="TextSearch", version="0.12"),
    PackageSpec(name="Plots"),
    PackageSpec(name="KNearestCenters"),
    PackageSpec(name="HypertextLiteral"),
    PackageSpec(name="CSV"),
    PackageSpec(name="DataFrames"),
    PackageSpec(name="Embeddings", version="0.4")
])

using SimilaritySearch, TextSearch, Plots, KNearestCenters, LinearAlgebra, Embeddings, HypertextLiteral, CSV, DataFrames, Base64, Random
using Downloads: download

  Activating project at `~/IR-2022/Unidades`


In [2]:
]status

      Status `~/IR-2022/Unidades/Project.toml`
  [944b1d66] CodecZlib v0.7.0
  [ac1192a8] HypertextLiteral v0.9.4
  [b20bd276] InvertedFiles v0.4.1 `../../Research/InvertedFiles.jl`
  [682c06a0] JSON v0.21.3
  [4dca28ae] KNearestCenters v0.7.1
  [8ef0a80b] Languages v0.4.3
  [eb30cadb] MLDatasets v0.7.3
  [053f045d] SimilaritySearch v0.9.4 `../../Research/SimilaritySearch.jl`
  [fb8f903a] Snowball v0.1.0
  [2913bbd2] StatsBase v0.33.18
  [7f6f6c8a] TextSearch v0.12.5 `../../Research/TextSearch.jl`


In [3]:
function knn(index, labels, q, k)
    res = KnnResult(k)
    search(index, q, res)
    mode(labels[idview(res)])
end

function mymode(c, labels, f)
    n = length(c)
    empty!(f)
    for id in c
        id == 0 && break  # searchbatch stores zeros at the end of the result when the result set is smaller than the required one
        l = labels[id]
        f[l] = get(f, l, 0) + 1
    end
    
    if length(f) == 0
        rand(labels)
    else
        argmax(last, f) |> first
    end
end

function knn(I, labels)
    f = Dict{eltype(labels), Int}()
    [mymode(c, labels, f) for c in eachcol(I)]
end

function scores(gold, pred)
    s = classification_scores(gold, pred)
    (macrof1=s.macrof1, macrorecall=s.macrorecall, accuracy=s.accuracy)
end

scores (generic function with 1 method)

In [4]:
struct ValidVocabulary{DictType} <: AbstractTokenTransformation
    valid::DictType
end

TextSearch.transform_unigram(S::ValidVocabulary, tok) = haskey(S.valid, tok) ? tok : nothing

function text_model_and_vectors(
        corpus, vocab;
        localweighting=TpWeighting(),
        globalweighting=BinaryGlobalWeighting(),
        nlist=[1],
        qlist=[],
        slist=[],
        group_usr=true,
        group_url=true,
        group_num=true,
        del_diac=true,
        lc=true
    )

    voc = Vocabulary(length(corpus))
    for v in vocab
       push!(voc, v)
    end
    
    tt = ValidVocabulary(voc.token2id)
    textconfig = TextConfig(; group_usr, group_url, del_diac, lc, group_num, nlist, qlist, slist, tt)
    tokenize_and_append!(voc, textconfig, corpus)
    model = VectorModel(globalweighting, localweighting, voc)
    (; textconfig, model, voc)
end


text_model_and_vectors (generic function with 1 method)

In [18]:
function embeddings(embname="MX.vec")
    embfile = "../data/$embname"
    !isfile(embfile) && download("http://geo.ingeotec.mx/~sadit/regional-spanish-models/$embname", embfile)
    emb = load_embeddings(FastText_Text, embfile)  # you can change with any of the available embeddings in `Embeddings`
    for c in eachcol(emb.embeddings)
        normalize!(c)
    end
    
    (; X=emb.embeddings, vocab=emb.vocab)
end

function vectorize_as_prototype!(c, text, E, T)
    x = vectorize(T.model, T.textconfig, text; normalize=false)
    if length(x) == 1 && haskey(x, 0)
        #@warn "empty vector $(Int(i)) selecting a random vector for it " # $(corpus[i])
        rand!(c)
    else
        for (id, weight) in x
            id == 0 && continue
             c .= c .+ weight .* view(E.X, :, id)
        end
    end
    
    normalize!(c)
end

function vectorize_corpus_as_prototypes(corpus, E, T)
    dim = size(E.X, 1)
    n = length(corpus)
    C = zeros(Float32, dim, n)
    # Threads.@threads
    for i in 1:n
        vectorize_as_prototype!(view(C, :, i), corpus[i], E, T)
    end
    
    C
end

function create_index(db, recall)
    dist = NormalizedCosineDistance()
    index = SearchGraph(; dist, db, verbose=false)
    index!(index; callbacks=SearchGraphCallbacks(MinRecall(recall)))
    optimize!(index, MinRecall(recall))
    index
end

create_index (generic function with 2 methods)

In [16]:
@time E = embeddings()
@time D = CSV.read("../data/spanish-news-in-twitter.csv", DataFrame; delim=',', missingstring="XXXX")
@time D[:, :text] = String.(base64decode.(D.text))
@time T = text_model_and_vectors(D.text, E.vocab)
size(D.text), size(E.X), size(E.vocab)

 35.425357 seconds (10.15 M allocations: 11.600 GiB, 3.70% gc time, 0.02% compilation time)
  0.033548 seconds (50.38 k allocations: 15.346 MiB)
  0.090820 seconds (1.25 M allocations: 100.733 MiB)
 10.854681 seconds (5.35 M allocations: 321.371 MiB)


((41999,), (300, 438136), (438136,))

In [17]:
@time C = vectorize_corpus_as_prototypes(D.text, E, T)

  1.788759 seconds (3.54 M allocations: 340.191 MiB, 7.17% compilation time)


300×41999 Matrix{Float32}:
 -0.0339036   -0.0428609   -0.0670109    …  -0.0374637   -0.0475409
  0.0265874    0.0223019    0.00263203       0.0131571    0.018139
 -0.0370614    0.00167701  -0.0378757       -0.0102305    0.0157182
  0.0181526    0.0105444   -0.0240541       -0.0646235    0.0214401
  0.0134366    0.0189434    0.0366603       -0.0527604   -0.00279745
 -0.0213551   -0.0188397    0.0460011    …   0.0460707    0.0460432
  0.0239996   -0.018044    -0.000361117     -0.00433237   0.0459184
  0.0195599    0.0119689    0.00421374       0.0138044   -0.0359088
  0.0655299    0.0465704    0.110764         0.0814923    0.0444924
 -0.00386314   0.0205548   -0.0155296        0.0353385   -0.00603773
  0.0261242    0.0328563    0.0815982    …   0.0578312    0.0652253
 -0.0106883    0.0192735    0.0482066        0.00723559   0.106062
  0.0287057    0.0271132    0.0213498        0.0690113    0.0219228
  ⋮                                      ⋱               
 -0.0379055   -0.0606254   -0.0

### Se crea el índice métrico

In [19]:
@time index = create_index(MatrixDatabase(C), 0.95)

  1.369040 seconds (630.47 k allocations: 84.675 MiB, 2.79% compilation time)


SearchGraph{NormalizedCosineDistance, MatrixDatabase{Matrix{Float32}}, BeamSearch}
  dist: NormalizedCosineDistance NormalizedCosineDistance()
  db: MatrixDatabase{Matrix{Float32}}
  links: Array{Vector{Int32}}((41999,))
  locks: Array{Base.Threads.SpinLock}((41999,))
  hints: Array{Int32}((108,)) Int32[23, 66, 83, 153, 197, 210, 230, 343, 429, 441  …  2880, 2955, 3005, 3035, 3100, 3101, 3122, 3148, 3153, 3156]
  search_algo: BeamSearch
  verbose: Bool false


### Búsqueda de todos los vecinos cercanos en el vocabulario, observe la conveniencia del uso de un índice

Entre más sean las consultas más se ve la bondad (~42k consultas)

In [20]:
let k = 32
    t1 = @elapsed I, _ = allknn(index, k)
    # WARNING: Don't run the following line, it takes too much time
    t2 = @elapsed gI, _ = allknn(ExhaustiveSearch(; db=index.db, dist=index.dist), k)
    r = macrorecall(gI, I)
    n = size(I, 2)
    labels = String.(D.screen_name)
    s1 = scores(labels, knn(I, labels))
    s2 = scores(labels, knn(gI, labels))
    
    @htl """
    <div>macro-recall: $r, n: $n</div>
    <div>searchgraph search time: $t1, scores: $s1</div>
    <div>brute force search time: $t2, scores: $s2</div>
    """
end

┌ Info: precision is zero for label 'fabriziomejia'; #classes=68 
└ @ KNearestCenters /home/sadit/.julia/packages/KNearestCenters/katlq/src/scores.jl:163
┌ Info: precision is zero for label 'lydiacachosi'; #classes=68 
└ @ KNearestCenters /home/sadit/.julia/packages/KNearestCenters/katlq/src/scores.jl:163
┌ Info: precision is zero for label 'Adela_Micha'; #classes=68 
└ @ KNearestCenters /home/sadit/.julia/packages/KNearestCenters/katlq/src/scores.jl:163
┌ Info: precision is zero for label 'patriciajaniot'; #classes=68 
└ @ KNearestCenters /home/sadit/.julia/packages/KNearestCenters/katlq/src/scores.jl:163
┌ Info: precision is zero for label 'laoctava_tv'; #classes=68 
└ @ KNearestCenters /home/sadit/.julia/packages/KNearestCenters/katlq/src/scores.jl:163
┌ Info: precision is zero for label 'jrisco'; #classes=68 
└ @ KNearestCenters /home/sadit/.julia/packages/KNearestCenters/katlq/src/scores.jl:163
┌ Info: precision is zero for label 'JorgeGCastaneda'; #classes=68 
└ @ KNearestCenters

<div>macro-recall: 0.9572593990333104, n: 41999</div>
<div>searchgraph search time: 1.018351672, scores: <span class="Core-NamedTuple">(macrof1 = 0.19272089103918924, macrorecall = 0.18219208377706542, accuracy = 0.30334055572751734)</span></div>
<div>brute force search time: 7.205663461, scores: <span class="Core-NamedTuple">(macrof1 = 0.20031139492836822, macrorecall = 0.18713400494194024, accuracy = 0.31243601038119956)</span></div>

### Búsqueda y presentación de los resultados

In [21]:
function search_and_display(index, qtext, k, D, E, T)
    res = KnnResult(k)
    q = zeros(Float32, size(E.X, 1))
    vectorize_as_prototype!(q, qtext, E, T)
    @time search(index, q, res)
    
    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$id</td><td>$(round(d, digits=3))</td> <td>$(D.screen_name[id])</td><td> $(D.text[id])</td> </tr>")
    end

    display(@htl """<h2>resultados for "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


search_and_display (generic function with 1 method)

In [22]:

display(@htl "<h1>Ejemplos de búsqueda</h1>")
search_and_display(index, "el gobierno de andres manuel lopez", 7, D, E, T)
search_and_display(index, "trafico de drogas", 7, D, E, T)
search_and_display(index, "covid corona virus", 7, D, E, T)


<h1>Ejemplos de búsqueda</h1>

  0.000384 seconds (4 allocations: 1.844 KiB)


1,26295,0.113,Radio_Formula,El presidente Andrés Manuel López Obrador exhibió el supuesto sueldo de Loret de Mola... con un documento con errores ortográficos. https://t.co/nx4bDFfw8p
2,40774,0.114,El_Universal_Mx,Sigue lo más relevante de la conferencia del presidente Andrés Manuel López Obrador ⬇️ https://t.co/pA4oo1iCbU
3,41634,0.116,PaolaRojas,"RT @nmas: El presidente de México, Andrés Manuel López Obrador, habló en su conferencia mañanera sobre el presunto cobro de derecho de piso…"
4,21913,0.122,CNNEE,Así pensó un plan B el presidente de México Andrés Manuel López Obrador para organizar la consulta de revocación de mandato. https://t.co/GaAeHdjhAG
5,40401,0.125,RicardoAlemanMx,RT @ContrapesoC: El expresidente Vicente Fox se lanzó contra el presidente Andrés Manuel López Obrador por sus declaraciones y por la inseg…
6,8834,0.126,azucenau,Informa @RicardoAnayaC que ha sido citado en el Reclusorio Norte por lo que acusa persecución de parte del presidente Andrés Manuel López Obrador
7,24542,0.127,azucenau,"#ALMOMENTO | Por segundo día consecutivo, el presidente Andrés Manuel López Obrador salió esta tarde de Palacio Nacional. https://t.co/tdBqBYhhuc"


  0.001109 seconds (6 allocations: 7.594 KiB)


1,24024,0.245,LaRazon_mx,#SSPC: Red de tráfico de migrantes es ubicada en #Guanajuato y #Tamaulipas https://t.co/2TdiV3G2Zf https://t.co/hctIUWIx7G
2,902,0.263,AristeguiOnline,El comercio ilegal de drogas se digitaliza con la pandemia https://t.co/dzZtvcjBAA https://t.co/hrG2x5a5xs
3,27495,0.267,CiroGomezL,"Mil 500 elementos de la @SEMAR_mx llegaron al AICM para tomar el control de la seguridad. Los marinos realizarán tareas de vigilancia, inspección, apoyo y control para evitar el tráfico de armas, drogas, divisas, mercancía ilegal y hasta el tráfico de personas: https://t.co/85QYwB26l8"
4,32075,0.274,TelemundoNews,🚗 Los Ángeles registra un aumento de accidentes de tráfico con conductores que se dan a la fuga. https://t.co/NROOouz3cg
5,14581,0.276,DoliaEstevez,"El apabullante poder de fuego de los cárteles Jalisco Nueva Generación y Sinaloa es testimonio del rotundo fracaso de las políticas de ambos gobiernos para controlar el tráfico ilegal de armas, escribo. https://t.co/RGeSrHuEcL"
6,29417,0.28,El_Universal_Mx,RT @ElUniversal_SLP: #Entérate ⛔️ Los trabajos de repavimentación de la carretera 57 provocan filas kilométricas de vehículos desde tempran…
7,22266,0.281,TelemundoNews,Realizan uno de los mayores decomisos de droga en #ElSalvador. https://t.co/fXG1vFysK0


  0.001133 seconds (6 allocations: 7.594 KiB)


1,37494,0.462,abc_es,Primeros datos de la eficacia de dos antivirales frente al virus de la viruela del simio https://t.co/TdBHvIIT3w
2,21303,0.463,bbcmundo,"El virus no solo nos enferma, también borra nuestras defensas contra otras enfermedades https://t.co/b1xfDZdfQS"
3,31767,0.468,AristeguiOnline,Molnupiravir elimina el virus SARS-CoV-2 activamente infeccioso al tercer día de comenzar la terapia: estudio https://t.co/93yBkWlj3G
4,24559,0.476,El_Universal_Mx,Es el primer deceso registrado en la entidad por esa variante del virus SARS-CoV-2 https://t.co/3FlnzSNpbk
5,16410,0.477,bbcmundo,El alentador efecto de la vacuna contra el virus del papiloma humano (VPH) en la reducción del cáncer de cuello uterino https://t.co/rQV9L9Q9F4
6,38128,0.482,SinEmbargoMX,Fármacos contra enfermedad inflamatoria intestinal protegerían de la COVID grave https://t.co/b7yXzIGahI https://t.co/nGZVBpX9Xi
7,30761,0.484,TelemundoNews,⚠️ 💊 @Pfizer retira del mercado algunos medicamentos contra la hipertensión por posible riesgo de cáncer. Otros medicamentos genéricos también fueron sacados del mercado. https://t.co/JJ3oac4Gek


In [23]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>")

for i in 1:3
    for qid in rand(1:length(D.text))
        search_and_display(index, D.text[qid], 7, D, E, T)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>

  0.000621 seconds (4 allocations: 1.844 KiB)


1,17985,-0.0,SinEmbargoMX,"RT @MagazineSEMX: Saúl ""Canelo"" Álvarez se aventura en el peso crucero y reta al campeón Ilunga Makabu https://t.co/9kftm5JWLL https://t.co…"
2,19463,0.101,El_Universal_Mx,"RT @ElUniversalOax: 🥊 Sergio Chirino Jr. (@chirino_oax), el joven boxeador de 27 años sueña con traer el triunfo y un campeonato a Oaxaca;…"
3,38678,0.101,Milenio,"RT @laaficion: Devin Haney, el boxeador que se forjó en un bar de Tijuana 🥊 https://t.co/U5JHSzHTvG https://t.co/XgVRdcHz8b"
4,37268,0.104,EFEnoticias,"RT @EFEdeportes: 🏉 Patricia García, en su retirada: ""El rugby me lo ha dado todo, me ha forjado como persona"". 🔝 @COE_Presidente: ""Su lega…"
5,32691,0.106,PublimetroMX,RT @Publisport_MX: ¡Hay tiro! El César del Boxeo se enfrentará a Daniel Zaragoza en la Plaza de Toros México 🥊 @IvanNavarro1006 te lo cuen…
6,16718,0.107,CNNEE,"RT @deportescnn: El ""Canelo"" Álvarez podría ser considerado como el mejor boxeador mexicano de la historia https://t.co/6Ufwiq6jGs https://…"
7,16482,0.108,El_Universal_Mx,RT @UnivDeportes: VA POR UNA HAZAÑA 🇲🇽 Saúl 'Canelo' Álvarez se podría convertir en el campeón absoluto supermediano y así escribir su nom…


  0.000637 seconds (4 allocations: 1.844 KiB)


1,34497,0.0,abc_es,En 'cosas de rojos' https://t.co/59xmCPsJLp ✍ José Sacristán rinde homenaje en ABC a su amigo y colega de profesión Juan Diego
2,27211,0.1,Milenio,Carlos Cabrera se reencuentra con Hugo Savinovich en Triple A después de 11 años El narrador aprovechó su presentación para rendir un homenaje a Arturo ‘Rudo’ Rivera y Super Muñeco https://t.co/Sk2Bom8KXL https://t.co/8JIRVOyQ34
3,19851,0.104,azucenau,"En agosto pasado, Issa Osorio, periodista de #Telemundo, entrevistó en la prisión a Miguel Ángel Félix Gallardo, ""el jefe de jefes"" ¿Podría continuar su condena en su hogar? Estoy conversando con @osorioissa en #AzucenaxFórmula"
4,8109,0.105,TelemundoNews,Miles de fanáticos de Vicente Fernández usan las redes sociales para mostrarle sus deseos de recuperación tras el accidente que sufrió en su rancho. 👉🏻 El artista Roberto Márquez pintó un mural en homenaje al intérprete de rancheras. https://t.co/qHI0wGBVsQ
5,21544,0.107,El_Universal_Mx,"RT @universalqro: ""El Payo"", Jerónimo, José Mauricio, José Funtanet y los forjados Amadores de México estarán en la plaza de toros Santa Ma…"
6,36276,0.109,Milenio,"▶ Familiares y amigos velan los restos de Yolanda Martínez, joven desaparecida en #NuevoLeón; el padre de la víctima asegura que el caso es un feminicidio 📺 Los detalles de Pedro Delgado con @danielhsaldivar y @paamvillanueva en #MILENIO18h https://t.co/Ab067jkXSE"
7,24567,0.109,AdriDelgadoRuiz,"NUEVO LIBRO: *”Crónicas Transatlánticas: De batallas, amores y otros exilios""* del Gran productor y escritor #LuisdeLlano. Es un recuento de la historia de su padre don Luis de Llano Palmer,quien llegó a México para crear y transformar la historia como se entiende hoy en día🇲🇽🇪🇸 https://t.co/oyb8Db57uZ"


  0.000519 seconds (4 allocations: 1.844 KiB)


1,25570,0.0,UniNoticias,🎥 El Pentágono advierte que Rusia podría simular un ataque de Ucrania para justificar una invasión. ✅ Sigue esta y más noticias las 24 horas en @PrendeTV. https://t.co/vBtWSUwcX8
2,31891,0.045,UniNoticias,🎥 “El conflicto está lejos de terminar”: experto advierte sobre posibles futuras maniobras de Rusia en Ucrania. ✅ Sigue esta y más noticias las 24 horas en @vixdotcom. #ViX #ViXStreaming #ViXGratis https://t.co/xsrCdfPxW5
3,30370,0.048,UniNoticias,"🎥 El ataque contra el exbeibolista David Ortiz habría sido ordenado por un narcotraficante, según autoridades de República Dominicana. ✅ Sigue esta y más noticias las 24 horas en @PrendeTV. https://t.co/Gyvpq7erkT"
4,32725,0.049,UniNoticias,"🎥 Erich de la Fuente, analista político, dijo que es importante que occidente ayude masivamente al ejército ucraniano. ✅ Sigue esta y más noticias las 24 horas en @VIX. #ViX #ViXStreaming #ViXGratis https://t.co/1Mut5Q35Or"
5,34433,0.053,UniNoticias,"Trevor Reed estuvo detenido en Rusia desde el año 2019 y este miércoles fue liberado. Para lograrlo, el gobierno de EEUU tuvo que dejar libre a Konstantin Yaroshenko. ✅ Sigue esta y más noticias las 24 horas en @VIX. #ViX #ViXStreaming #ViXGratis 👉 https://t.co/bj1kgzgh5v. https://t.co/9oIzsVlaLb"
6,31368,0.053,UniNoticias,🎥 Rusia anunció que reducirán sus actividades militares con el fin de facilitar las conversaciones con Ucrania. ✅ Sigue esta y más noticias las 24 horas en @PrendeTV https://t.co/o8bNE5n6kj
7,23483,0.054,UniNoticias,🎥 Más de 30 senadores le enviaron una carta a la administración del presidente Joe Biden en la que le piden un nuevo TPS para los centroamericanos. ✅ Sigue esta y más noticias las 24 horas en @PrendeTV. https://t.co/r1n1x93dGK


# Actividades
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- ¿Cuál sería el símil de bigramas y trigramas para este esquema de representación semántica? Implementelo.
- Defina $H^{pt}_d$, donde $pt$ es la probabilidad de ocurrencia de término.- Defina $H^{pt}_d$, donde $pt$ es la probabilidad de ocurrencia de término.
- Implemente $H^\textsf{TF}_d$ y $H^\textsf{pt}_d$ de manera secuencial y con un índice métrico. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. El reporte deberá contener un ensayo de [@KSKW2015] como introducción. Reporte los resultados de sus implementaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
